In [42]:
import json


# Open the file
with open('Reentrancy_AutoExtract_fullnodes.json', 'r') as f:
  # Create an empty list
  lines = []
  # Iterate over the lines in the file
  for line in f:
    # Strip the line of leading and trailing whitespace
    line = line.strip()
    # Add the line to the list
    line = json.loads(line[:len(line)-1])
    lines.append(line)


In [32]:
import numpy as np
x = np.arange(start=1, stop=64, step=1)
print(x[0:11])
print(x[11:22])
print(x[22:32])


[ 1  2  3  4  5  6  7  8  9 10 11]
[12 13 14 15 16 17 18 19 20 21 22]
[23 24 25 26 27 28 29 30 31 32]


In [ ]:
from typing import List
def onehot2index(vec: List[int]):
    return vec.index(1)


def return_connected_nodes(vec, index):
    connected = []
    # Start counting from 1, to avoid index for connection to NULL
    for i in range(1,len(vec)):
        if vec[i] > 0:
            connected.append(i)
    return connected


In [53]:
import networkx as nx

X = []
A = []
y = []

for data in lines:
    indexes = []
    x_i = []
    a_i = []
    y_i = int(data["target"])


    graph = data["graph"] # THIS IS THE EDGE DATA
    node_features = data["node_features"] # THIS IS EQUAL TO # OF NODES
    contract_name = data["contract_name"]

    nodes = set()
    for edge in graph:
        nodes.add(edge[2])
        nodes.add(edge[0])

    node_num = len(node_features)
    # print(contract_name, nodes, len(nodes), graph)
    for i in range(node_num):
        feat = node_features[i]
        node_type = feat[0:11]
        onehot_index = onehot2index(node_type)
        indexes.append(onehot_index)

    for i in range(node_num):
        feat = node_features[i]
        # Extract features as a vector (timestamp)
        """
        node_type = feat[0:11]
        adj_list = feat[11:22]
        execution_order = feat[22:32]
        node_op = feat[32:36]
        other = feat[36:]
        error_check = sum(other)
        """

        # Extract features as a vector (reentrancy)
        node_type = feat[0:11]
        ac_type = feat[11:14]
        adj_list = feat[14:25]
        execution_order = feat[25:35]
        node_op = feat[35:39]
        other = feat[39:]
        error_check = sum(other)

        connected_nodes = return_connected_nodes(adj_list)
        adj_row = [0 for j in range(len(indexes))]
        a_i.append(adj_row)

    y.append(y_i)



In [10]:
equal = 0
less = 0
more = 0
dupe = 0
one_target = 0
always_true = True

recognized_contracts = set()
print(lines[0])
for data in lines:
    graph = data["graph"] # THIS IS THE EDGE DATA
    node_features = data["node_features"] # THIS IS EQUAL TO # OF NODES
    contract_name = data["contract_name"]
    
    # feat_dims = list(map(lambda x: len(x)==64, node_features))
    # if not all(feat_dims):
    #     print("ERROR")
    if data["targets"]=="1":
        one_target += 1
    
    tmp = set()
    for edge in graph:
        tmp.add(edge[2])
        tmp.add(edge[0])
    # FOUND THE CONST
    if len(tmp) != len(node_features):
        always_true = False
        print("HUDHUSH", len(tmp),tmp)

    print(contract_name, len(node_features), graph)

    if contract_name in recognized_contracts:
        dupe += 1

    recognized_contracts.add(contract_name)
print(equal, less, more)
print(one_target)
print(dupe)
print(always_true)

{'targets': '0', 'graph': [[0, 9, 1], [0, 9, 2], [1, 2, 2], [2, 7, 3]], 'contract_name': '15.sol', 'node_features': [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
15.sol 4 [[0, 9, 1], [0, 9, 2], [1, 2, 2], [2, 7, 3]]
42.sol 5 [[0, 9, 1], [0, 9, 2], [1, 2, 2], [2, 1, 3]

In [11]:
nodelist = ['NULL', 'N1', 'N2', 'N3', 'N4', 'N5', 'C1', 'C2', 'C3', 'C4', 'C5']
node_sentence = []

for i in range(len(nodelist)):
    node_sentence.append(i + 1)
print(node_sentence)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [12]:
node_dict = dict(zip(nodelist, node_sentence))
node_dict

{'NULL': 1,
 'N1': 2,
 'N2': 3,
 'N3': 4,
 'N4': 5,
 'N5': 6,
 'C1': 7,
 'C2': 8,
 'C3': 9,
 'C4': 10,
 'C5': 11}

In [13]:
import numpy as np
node_vectors = {}
for word, index in node_dict.items():
    node_array = np.zeros(len(nodelist), dtype=int)
    node_vectors[word] = node_array
    node_vectors[word][index - 1] = 1.0

# Vectors of length 11
for k,v in node_vectors.items():
    print(k,v)


NULL [1 0 0 0 0 0 0 0 0 0 0]
N1 [0 1 0 0 0 0 0 0 0 0 0]
N2 [0 0 1 0 0 0 0 0 0 0 0]
N3 [0 0 0 1 0 0 0 0 0 0 0]
N4 [0 0 0 0 1 0 0 0 0 0 0]
N5 [0 0 0 0 0 1 0 0 0 0 0]
C1 [0 0 0 0 0 0 1 0 0 0 0]
C2 [0 0 0 0 0 0 0 1 0 0 0]
C3 [0 0 0 0 0 0 0 0 1 0 0]
C4 [0 0 0 0 0 0 0 0 0 1 0]
C5 [0 0 0 0 0 0 0 0 0 0 1]
